## Final Project Submission

Please fill out:
* Student name: Steven Rosa
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL:


### Work flows
#### Parrish
https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469

#### Graph convolutional network ("semi-supervised")
https://towardsdatascience.com/text-based-graph-convolutional-network-for-semi-supervised-bible-book-classification-c71f6f61ff0f

In [1]:
url = 'https://www.gutenberg.org/ebooks/4217'


import numpy as np
import pandas as pd
import math

from numpy import dot #parrish
from numpy.linalg import norm #parrish
import en_core_web_sm #parrish
import spacy #parrish 
from __future__ import unicode_literals #parrish

# Learn workflow 

In [ ]:
#Remove stopwords: 
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')

# It is generally a good idea to also remove punctuation
import string

# Now we have a list that includes all english stopwords, as well as all punctuation
stopwords_list += list(string.punctuation)

#https://learn.co/tracks/data-science-career-v1-1/module-4-advanced-machine-learning-deep-learning/section-37-foundations-of-natural-language-processing-nlp/feature-engineering-for-text-data#

#&c &c &c

# Parrish workflow

In [2]:
nlp = spacy.load('en_core_web_md')

In [6]:
with open('chapter1.txt', 'r', encoding = 'utf-8') as f1, \
    open('chapter2.txt', 'r', encoding = 'utf-8') as f2, \
    open('chapter3.txt', 'r', encoding = 'utf-8') as f3, \
    open('chapter4.txt', 'r', encoding = 'utf-8') as f4, \
    open('chapter5.txt', 'r', encoding = 'utf-8') as f5:
    ch1 = nlp(f1.read())
    ch2 = nlp(f2.read())
    ch3 = nlp(f3.read())
    ch4 = nlp(f4.read())
    ch5 = nlp(f5.read())

tokens1 = list(set([w.text for w in ch1 if w.is_alpha]))
tokens2 = list(set([w.text for w in ch2 if w.is_alpha]))
tokens3 = list(set([w.text for w in ch3 if w.is_alpha]))
tokens4 = list(set([w.text for w in ch4 if w.is_alpha]))
tokens5 = list(set([w.text for w in ch5 if w.is_alpha]))

In [17]:
#From Parrish
nlp.vocab['cheese'].vector

In [7]:
#Functions from Parrish


def subtractv(coord1, coord2):
    return [c1 - c2 for c1, c2 in zip(coord1, coord2)]

def addv(coord1, coord2):
    return [c1 + c2 for c1, c2 in zip(coord1, coord2)]

def meanv(coords):
    # assumes every item in coords has same length as item 0
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean


#Convenience function
def vec(s):
    return nlp.vocab[s].vector

# cosine similarity
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0
    
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine(vec_to_check, vec(x)),
                  reverse=True)[:n]

def sentvec(s):
    sent = nlp(s)
    return meanv([w.vector for w in sent])

def spacy_closest_sent(space, input_str, n=10):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                  reverse=True)[:n]

In [14]:
cosine(vec('dog'), vec('puppy')) > cosine(vec('trousers'), vec('pants'))

True

In [13]:
cosine(vec('dog'), vec('puppy')), cosine(vec('trousers'), vec('pants'))

(0.85852146, 0.8541468)

In [13]:
#Parrish
spacy_closest(tokens1, vec("pie"))

['pudding',
 'bread',
 'chocolate',
 'butter',
 'buttered',
 'sauce',
 'cream',
 'shortbread',
 'tasty',
 'hearty']

In [14]:
spacy_closest(tokens2, vec("pie"))

['bread',
 'chocolate',
 'bake',
 'butter',
 'hearty',
 'gravy',
 'apples',
 'greased',
 'filling',
 'saucer']

In [15]:
spacy_closest(tokens5, vec("pie"))

['cake',
 'recipe',
 'bread',
 'chocolate',
 'crusts',
 'cakes',
 'pancakes',
 'biscuit',
 'bacon',
 'risotto']

In [37]:
ch1_list = [w.text for w in ch1 if w.is_alpha]

In [39]:
len(ch1_list), len(set(ch1_list))

(17393, 2486)

In [40]:
#Need to remove stop words
pd.Series(ch1_list).value_counts()

the            1155
and             774
of              430
to              391
was             383
a               337
he              326
his             323
in              300
had             222
said            205
that            196
He              172
it              167
him             142
on              136
were            133
not             119
with            108
you             107
for             104
they             98
The              98
at               95
be               91
Mr               84
out              84
would            83
up               80
It               76
               ... 
generals          1
soon              1
flowerbeds        1
twined            1
reply             1
inclined          1
Xavier            1
Lavender          1
guards            1
arguing           1
reddened          1
finger            1
dropped           1
backhand          1
pinned            1
chest             1
crashing          1
poor              1
hushed            1


In [46]:
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens5, addv(blue_to_sky, vec("grass")))

['grass',
 'green',
 'Green',
 'yellow',
 'Yellow',
 'red',
 'lawn',
 'blue',
 'leaf',
 'brown']

In [56]:

sents1 = list(ch1.sents)

for sent in spacy_closest_sent(sents1, "I went to church this morning."):
    print(sent.text)
    print("---")

I’d go
straight up to the rector and tell him about it after dinner.


---
—Father Dolan came in today and pandied me because I was not writing my
theme.


---
Dante turned on her and said:

—And am I to sit here and listen to the pastors of my church being
flouted?


---
I’d go up and
tell the rector on him.


---
But he said:

—Gentlemen, the happiest day of my life was the day on which I made my
first holy communion.


---
There was a cold night smell in the chapel.
---
It was lovely to be tired.
---
Brother Michael was very decent and always told him the news out of the
paper they got every day up in the castle.
---
That never went to mass.


---
And
that night Mr Casey had not gone to Dublin by train but a car had come
to the door
---


# Insight workflow
### Data cleaning:
- tokenize
- remove non-alphanumeric
- to lower case
- combine words with similar spellings
- lemmatize?

### Words into numbers
- bag words [0 0 0 0 0 1 0 0 ]